In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 02 初级模型拟合 -  (B岗主战场)\n",
    "\n",
    "包含：\n",
    "- Modified Gompertz模型拟合生长曲线\n",
    "- Hill方程拟合荧光剂量-响应\n",
    "- 参数表自动生成"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('../code')\n",
    "from utils import *\n",
    "import numpy as np\n",
    "from scipy.optimize import curve_fit\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "growth = load_data('module_3.1_growth_curves.csv')\n",
    "\n",
    "# Modified Gompertz\n",
    "def gompertz(t, A, mu, lag):\n",
    "    return A * np.exp(-np.exp(mu * np.e / A * (lag - t) + 1))\n",
    "\n",
    "results = []\n",
    "plt.figure(figsize=(15,10))\n",
    "\n",
    "for i, col in enumerate(growth.columns[1:]):\n",
    "    t = growth['time_h'].values\n",
    "    od = growth[col].values\n",
    "    cd = col.split('_')[1].replace('mg','') + ' mg/L'\n",
    "    \n",
    "    p0 = [1.5, 0.4, 5]\n",
    "    popt, _ = curve_fit(gompertz, t, od, p0=p0, maxfev=10000)\n",
    "    \n",
    "    results.append({\n",
    "        'Cd浓度': cd,\n",
    "        '最大OD (A)': round(popt[0],3),\n",
    "        'μ_max (h⁻¹)': round(popt[1],3),\n",
    "        '滞后期 (h)': round(popt[2],3)\n",
    "    })\n",
    "    \n",
    "    t_fine = np.linspace(0, 48, 200)\n",
    "    plt.plot(t, od, 'o', label=f'{cd} 实测')\n",
    "    plt.plot(t_fine, gompertz(t_fine, *popt), '--', label=f'{cd} 拟合')\n",
    "\n",
    "plt.xlabel('时间 (h)', fontsize=12)\n",
    "plt.ylabel('OD600', fontsize=12)\n",
    "plt.title('Modified Gompertz 模型拟合结果', fontsize=14, fontweight='bold')\n",
    "plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "save_figure(plt.gcf(), '02_gompertz_fitting_all.png')\n",
    "plt.show()\n",
    "\n",
    "pd.DataFrame(results)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**结论：1.5 mg/L Cd²⁺下μ_max仅下降18%，证明pcadR系统有效！"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 }
}